In [1]:
#!/usr/bin/env python3

import numpy as np
import os.path as op
from pprint import pformat
import argparse
# EEG utilities
import mne
from mne.preprocessing import ICA, create_eog_epochs
from pyprep.prep_pipeline import PrepPipeline
# BIDS utilities
from mne_bids import BIDSPath, read_raw_bids
from util.io.bids import DataSink

# constants
BIDS_ROOT = '../data/bids'
DERIV_ROOT = op.join(BIDS_ROOT, 'derivatives')
FFR_PASSBAND = (100., 300.)
MICROSTATE_PASSBAND = (1., 30.)
TASK = 'expectations'
TMIN = -0.2
TMAX = 0.3

RAW_DIR = '../data/raw/' # where our data currently lives
BIDS_DIR = '../data/bids/' # where we want it to live
BAD_SUBS = []

sub = '11'
TASK = 'expectations'
run = '1'
'''
Parameters
----------
sub : str
    Subject ID as in BIDS dataset
'''
# load data
print('----------------- load data ------------------')
bids_path = BIDSPath(
    root = BIDS_ROOT,
    subject = sub,
    task = TASK,
    run = run,
    datatype = 'eeg'
    )
print(bids_path)
raw = read_raw_bids(bids_path, verbose = False)
events, event_ids = mne.events_from_annotations(raw)

# re-reference eye electrodes to become bipolar EOG
print('----------------- re-reference eye electrodes to become bipolar EOG ------------------')
raw.load_data()
def reref(dat):
    dat[0,:] = (dat[1,:] - dat[0,:])
    return dat
raw = raw.apply_function(
    reref,
    picks = ['leog', 'Fp2'],
    channel_wise = False
)
raw = raw.apply_function(
    reref,
    picks = ['reog', 'Fp1'],
    channel_wise = False
)

raw = raw.set_channel_types({'leog': 'eog', 'reog': 'eog'})

# run PREP pipeline (notch, exclude bad chans, and re-reference)
print('----------------- run PREP pipeline ------------------')
raw, events = raw.resample(int(4*FFR_PASSBAND[1]), events = events) # resample to 1200 Hz
np.random.seed(int(sub))
lf = raw.info['line_freq']
prep_params = {
    "ref_chs": "eeg",
    "reref_chs": "eeg",
    "line_freqs": np.arange(lf, FFR_PASSBAND[1], lf)
}
prep = PrepPipeline(
    raw,
    prep_params,
    raw.get_montage(),
    ransac = False,
    random_state = int(sub)
    )
prep.fit()

# Extract data from PREP
print('----------------- Extract data from PREP ------------------')
prep_eeg = prep.raw_eeg # get EEG channels from PREP
prep_non_eeg = prep.raw_non_eeg # get non-EEG channels from PREP
raw_data = np.concatenate((prep_eeg.get_data(), prep_non_eeg.get_data())) # combine data from the two

# Create info object for post-PREP data
print('Create info object for post-PREP data')
new_ch_names = prep_eeg.info['ch_names'] + prep_non_eeg.info['ch_names']
raw = raw.reorder_channels(new_ch_names) # modify the channel names on the original raw data
raw_info = raw.info # use the modified info from the original raw data object
 
# Combine post-prep data and new info
print('Create new raw object')
raw = mne.io.RawArray(raw_data, raw_info) # replace original raw object

## reconstruct the typical FFR/ABR montage we know and love
print('----------------- reconstruct the typical FFR/ABR montage ------------------')
raw_for_ffr = raw.copy().pick(['Cz', 'TP9', 'TP10']) # Cz and mastoids
raw_for_ffr.set_eeg_reference(ref_channels = ['TP9', 'TP10'])
raw_for_ffr = raw_for_ffr.pick(['Cz'])
# and filter (just the highpass, since lowpass applied when downsampling)
raw_for_ffr = raw_for_ffr.filter(l_freq = FFR_PASSBAND[0], h_freq = None)
# then epoch
epochs = mne.Epochs(
    raw_for_ffr,
    events,
    tmin = TMIN,
    tmax = TMAX,
    event_id = event_ids,
    baseline = (TMIN, 0.),
    preload = True
)

# Downsample
epochs = epochs.resample(sfreq = 5000)

# drop bad epochs
epochs.drop_bad(reject = dict(eeg = 35e-6))
# then save epochs for later
sink = DataSink(DERIV_ROOT, 'preprocess_ffr')
ffr_fpath = sink.get_path(
    subject = sub,
    task = TASK,
    run = run,
    desc = 'forFFR',
    suffix = 'epo',
    extension = 'fif.gz'
)
print(f'Saving epochs object to: {ffr_fpath}')
epochs.save(ffr_fpath, overwrite = True)

## now prepare non-epoched data for microstate jazz
# identify bad ICs on weakly highpassed data
print('----------------- prepare non-epoched data for microstate jazz ------------------')
raw_for_ica = raw.copy().filter(l_freq = 1., h_freq = None)
epochs_for_ica = mne.Epochs(
    raw_for_ica,
    epochs.events, # same events as FFR epochs
    tmin = TMIN,
    tmax = .0, # only prestim
    event_id = event_ids,
    baseline = None,
    preload = True
)

print('----------------- identify bad ICs on weakly highpassed data ------------------')
ica = ICA(n_components = 15, random_state = 0)
ica.fit(epochs_for_ica, picks = ['eeg', 'eog'])
eog_indices, eog_scores = ica.find_bads_eog(epochs_for_ica, threshold = 1.96)
ica.exclude = eog_indices

# filter to desired bandwidth and remove bad ICs
print('----------------- filter to desired bandwidth and remove bad ICs ------------------')
raw = raw.filter(*MICROSTATE_PASSBAND)
epochs_for_micro = mne.Epochs(
    raw,
    epochs.events, # same events as FFR epochs
    tmin = TMIN,
    tmax = .0, # only prestim
    event_id = event_ids,
    baseline = None,
    preload = True
)

# apply ICA
print('----------------- apply ICA ------------------')
ica.apply(epochs_for_micro) # transforms in place
# now we no longer need EOG channels
epochs_for_micro = epochs_for_micro.drop_channels('leog')
epochs_for_micro = epochs_for_micro.drop_channels('reog')
# and save
sink = DataSink(DERIV_ROOT, 'microstates')
micro_fpath = sink.get_path(
    subject = sub,
    task = TASK,
    run = run,
    desc = 'forMicrostate',
    suffix = 'epo',
    extension = 'fif.gz'
)

# Save data for microstate jazz
print(f'Saving epochs for microstate jazz to: {micro_fpath}')
epochs_for_micro.save(micro_fpath, overwrite = True)

# generate a report
print('----------------- generate a report ------------------')
report = mne.Report(verbose = True)
report.parse_folder(op.dirname(ffr_fpath), pattern = '*epo.fif.gz', render_bem = False)
if ica.exclude:
    fig_ica_removed = ica.plot_components(ica.exclude, show = False)
    report.add_figure(
        fig_ica_removed,
        title = 'Removed ICA Components',
        section = 'ICA'
    )
bads = prep.noisy_channels_original
html_lines = []
for line in pformat(bads).splitlines():
    html_lines.append('<br/>%s' % line)
html = '\n'.join(html_lines)
report.add_html(html, title = 'Interpolated Channels', section = 'channels')
report.add_html(epochs.info._repr_html_(), title = 'Epochs Info (FFR)', section = 'info')
report.add_html(epochs_for_micro.info._repr_html_(), title = 'Epochs Info (Microstates)', section = 'info')
report.save(op.join(sink.deriv_root, 'sub-%s.html'%sub), overwrite = True)

----------------- load data ------------------
../data/bids/sub-11/eeg/sub-11_task-expectations_run-1_eeg.vhdr
Used Annotations descriptions: ['11', '12', '13', '21', '22', '23', '31', '32', '33', '41', '42', '43', '51', '52', '53', '61', '62', '63']


/scratch/local/jobs/19200328/ipykernel_216929/651320076.py:47: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['leog', 'reog', 'Aux']
  raw = read_raw_bids(bids_path, verbose = False)
/scratch/local/jobs/19200328/ipykernel_216929/651320076.py:47: RuntimeWarning: Not setting positions of 3 misc/stim channels found in montage:
['leog', 'reog', 'Aux']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = read_raw_bids(bids_path, verbose = False)


----------------- re-reference eye electrodes to become bipolar EOG ------------------
Reading 0 ... 20891299  =      0.000 ...  4178.260 secs...


/scratch/local/jobs/19200328/ipykernel_216929/651320076.py:67: RuntimeWarning: The unit for channel(s) leog, reog has changed from NA to V.
  raw = raw.set_channel_types({'leog': 'eog', 'reog': 'eog'})


----------------- run PREP pipeline ------------------


KeyboardInterrupt: 

In [31]:
map_fp = '../data/captrak/pitch_tracking_64_at_FCZ.csv'
# map_fp
mapping_table = pd.read_csv(map_fp)
# mapping_table
mapping = {mapping_table.number[i]: mapping_table.name[i] for i in range(len(mapping_table))}
# mapping
raw.rename_channels(mapping)
# raw.add_reference_channels(ref_channels = ['FCz'])

ValueError: Invalid channel name(s) {'Ch17', 'Ch11', 'Ch5', 'Ch35', 'Ch2', 'Ch48', 'Ch62', 'Ch8', 'Ch38', 'Ch51', 'Ch43', 'Ch63', 'Ch36', 'Ch22', 'Ch20', 'Ch23', 'Ch15', 'Ch19', 'Ch33', 'Ch4', 'Ch21', 'Ch25', 'Ch9', 'Ch27', 'Ch41', 'Ch29', 'Ch7', 'Ch47', 'Ch18', 'Ch58', 'Ch45', 'Ch64', 'Ch3', 'Ch16', 'Ch60', 'Ch32', 'Ch59', 'Ch55', 'Ch10', 'Ch39', 'Ch61', 'Ch53', 'Ch6', 'Ch54', 'Ch50', 'Ch57', 'Ch40', 'Ch14', 'Ch46', 'Ch34', 'Ch13', 'Ch52', 'Ch26', 'Ch1', 'Ch30', 'Ch44', 'Ch31', 'Ch28', 'Ch42', 'Ch12', 'Ch56', 'Ch49', 'Ch37'} are not present in info

In [32]:
raw.ch_names

['Fp1',
 'Fz',
 'F3',
 'F7',
 'FT9',
 'FC5',
 'FC1',
 'C3',
 'T7',
 'TP9',
 'CP5',
 'CP1',
 'Pz',
 'P3',
 'P7',
 'O1',
 'Oz',
 'O2',
 'P4',
 'P8',
 'TP10',
 'CP6',
 'CP2',
 'Cz',
 'C4',
 'T8',
 'FT10',
 'FC6',
 'FC2',
 'F4',
 'F8',
 'Fp2',
 'leog',
 'AF3',
 'AFz',
 'F1',
 'F5',
 'FT7',
 'FC3',
 'C1',
 'C5',
 'TP7',
 'CP3',
 'P1',
 'P5',
 'PO7',
 'PO3',
 'POz',
 'PO4',
 'PO8',
 'P6',
 'P2',
 'CPz',
 'CP4',
 'TP8',
 'C6',
 'C2',
 'FC4',
 'FT8',
 'F6',
 'reog',
 'AF4',
 'F2',
 'Aux']

In [23]:
import re
import os
import itertools
from typing import Tuple, Iterator
KeyType = Tuple[str, str, str, str]

# def iter_raw_paths(data_dir) -> Iterator[KeyType]:
data_dir = '../data/raw/'
fnames = os.listdir(data_dir)

for fname in fnames:
    if 'sub' not in fname or '.vhdr' not in fname:
        continue

    # Get subject number
    filt = re.compile('(([0-9]|[1-9][0-9]){1,2})')
    sub = re.search(filt, fname).group(0)

    # Get task name
    task = 'expectations'

    # Get run number
    filt = re.compile('\w+[0-9]_([0-9]).*')
    run = re.findall(filt, fname)
    run = '1' if run == [] else run[0]

    key = (fname, sub, task, run)
    # print(key)
    # yield key


In [21]:
'.vhdr' not in fname

False